# Обработка Markdown и сохранение эмбеддингов в Qdrant

Этот ноутбук выполняет следующие шаги:

1. Парсит Markdown-файлы с вопросами и ответами.
2. Генерирует эмбеддинги с использованием модели `intfloat/multilingual-e5-large`.
3. Сохраняет эмбеддинги в векторную базу Qdrant для использования в ассистенте.

## Требования

- Установите зависимости: `pip install sentence-transformers qdrant-client`
- Запустите Qdrant: `docker run -p 6333:6333 qdrant/qdrant`
- Укажите путь к папке с Markdown-файлами в переменной `MD_DIRECTORY`.


In [ ]:
# Установка зависимостей (если требуется)
%pip install sentence-transformers qdrant-client

In [ ]:
# Импорт библиотек
import os
import re
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Конфигурация
MD_DIRECTORY = '../rag_files'  # Путь к папке с Markdown-файлами
COLLECTION_NAME = 'test_questions'  # Имя коллекции в Qdrant
MODEL_NAME = 'intfloat/multilingual-e5-large'  #модель для эмбеддингов
MODEL_PATH = './embendings_model' #путь до модели

## Парсинг Markdown-файлов

Функция `parse_md_files` читает Markdown-файлы и разделяет их на вопросы и ответы по заголовкам уровня 2 (`##`).


In [ ]:
def parse_md_files(directory: str) -> List[Dict[str, str]]:
    """
    Читает Markdown-файлы и разделяет их на вопросы и ответы.
    Возвращает список словарей с текстом, вопросом, ответом и метаданными.
    """
    chunks = []
    for filename in os.listdir(directory):
        if filename.endswith('.md'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # Разделяем на секции по заголовкам уровня 2 (##)
                sections = re.split(r'^##\s+', content, flags=re.MULTILINE)[1:]
                questions = [s.split('\n', 1)[0].strip() for s in sections]
                answers = [s.split('\n', 1)[1].strip() for s in sections]

                for i, (question, answer) in enumerate(zip(questions, answers)):
                    if question and answer:
                        chunks.append({
                            'question': question,
                            'answer': answer,
                            'text': f'Вопрос: {question}\nОтвет: {answer}',
                            'source': filename,
                            'chunk_id': f'{filename}_{i}',
                            'metadata': {'question': question}
                        })
    return chunks

# Выполняем парсинг и выводим первые 3 фрагмента для проверки
chunks = parse_md_files(MD_DIRECTORY)
for chunk in chunks[:3]:
    print(f"Chunk ID: {chunk['chunk_id']}, Question: {chunk['question'][:50]}..., Answer: {chunk['answer'][:50]}...")

## Загрузка модели для эмбендингов


In [ ]:
from sentence_transformers import SentenceTransformer

# Загружаем модель из Hugging Face
model = SentenceTransformer(MODEL_NAME)

# Сохраняем локально
model.save(MODEL_PATH)


## Генерация эмбеддингов и сохранение в Qdrant

Функция `vectorize_and_store` генерирует эмбеддинги и сохраняет их в Qdrant.


In [ ]:
import uuid

def vectorize_and_store(directory: str, collection_name: str):
    # Инициализация модели из локального пути
    model = SentenceTransformer(MODEL_PATH)

    # Инициализация Qdrant
    client = QdrantClient('localhost', port=6333)

    # Проверяем существование коллекции
    if not client.collection_exists(collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(size=1024, distance=models.Distance.COSINE)
        )

    # Получаем фрагменты
    chunks = parse_md_files(directory)

    # Генерируем эмбеддинги и сохраняем
    for chunk in chunks:
        embedding = model.encode(chunk['text'], convert_to_tensor=False).tolist()
        client.upsert(
            collection_name=collection_name,
            points=[
                models.PointStruct(
                    id=str(uuid.uuid4()),
                    vector=embedding,
                    payload={
                        'question': chunk['question'],
                        'answer': chunk['answer'],
                        'text': chunk['text'],
                        'source': chunk['source'],
                        'metadata': chunk['metadata']
                    }
                )
            ]
        )
    print(f'Сохранено {len(chunks)} фрагментов в коллекцию "{collection_name}"')

# Выполняем генерацию и сохранение
vectorize_and_store(MD_DIRECTORY, COLLECTION_NAME)

## Тестирование

Проверяем, как работает поиск по сохраненным эмбеддингам.


In [ ]:
# Тестовый поиск
model = SentenceTransformer(MODEL_NAME)
client = QdrantClient('localhost', port=6333)

query = 'Как отменить заказ?'
query_embedding = model.encode(query, convert_to_tensor=False).tolist()

search_results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_embedding,
    limit=3
)

for result in search_results:
    print(f"Score: {result.score:.4f}, Text: {result.payload['text'][:100]}...")